In [7]:
filename = "20200228_mouse_liver_13C_nontumor6 Analyte 3SRD19.txt"
nrows = 60
ncols = 120
formula = "C14H28O2"

polarity = -1
ppmThresh = 20
convSquare = 3
colormap = "gray"
threshold = True

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.rcParams['figure.dpi'] = 500
import scipy.stats as stats
import skimage.filters
from helper import *
import molmass
from PIL import Image

In [9]:
f = molmass.Formula(formula)
m0Mz = f.isotope.mass
adductChange = polarity * 1.0077276
comp = f.composition()
for row in comp:
    if row[0] == "C":
        numCarbons = int(row[1])

#load in data and organize
data = [r.strip().split() for r in open(filename,"r").readlines()[3:]]
data = {(x[0],float(x[1]),float(x[2])):{mz:float(i) for mz,i in zip(data[0],x[3:])} for x in data[1:] if len(x) > 0}
data = pd.DataFrame.from_dict(data,orient="index")
cols = data.columns.values
mzs = cols


#get mzs of interest
mzsOI = [m0Mz+adductChange+1.00336*x for x in range(numCarbons+1)]
mzMatchDict = {i:{"true":m,"observed":[],"matched":False} for m,i in zip(mzsOI,range(len(mzsOI)))}

#find matching peaks in data and output found and missing peaks
for m,i in zip(mzsOI,range(len(mzsOI))):
    matched = False
    for m2 in mzs:
        tmpM2 = float(m2)
        if abs(m-tmpM2) / m * 1e6 < ppmThresh:
            mzMatchDict[i]["observed"].append(m2)
            mzMatchDict[i]["matched"] = True
            matched = True
            break
    if not matched:
        print("missed",m)
    else:
        print("found",m,mzMatchDict[i]["observed"])


found 227.20120254158 ['227.2002']
found 228.20456254157997 ['228.2032']
missed 229.20792254158
found 230.21128254157998 ['230.2071']
found 231.21464254158 ['231.2130']
found 232.21800254157998 ['232.2166']
found 233.22136254158 ['233.2205']
found 234.22472254157998 ['234.2237']
found 235.22808254158 ['235.2269']
found 236.23144254157998 ['236.2302']
found 237.23480254158 ['237.2333']
found 238.23816254157998 ['238.2366']
found 239.24152254158 ['239.2402']
missed 240.24488254157998
missed 241.24824254158


In [10]:
#gather images for mzs of interest 
tensor = []
for mz in range(numCarbons+1):
    #iterate through mzs of interest
    if mzMatchDict[mz]["matched"]:   
        outarray = np.sum(np.array([getImage(data,x,nrows,ncols) for x in mzMatchDict[mz]["observed"]]),axis=0)
        nrows = len(outarray)
        ncols = len(outarray[0])
    else:
        outarray = np.zeros((nrows,ncols))
    tensor.append(outarray)

tensor = np.array(tensor)

image can only be downsampled


In [ ]:
#get thresholded image to define sample boundary

#create TIC image

#go through all features in dataset
if threshold:
    sumImage = np.sum([getImage(data,x,nrows,ncols) for x in mzs],axis=0)

    plt.imshow(sumImage,cmap=colormap)
    plt.colorbar()
    plt.figure()
    plt.hist(sumImage.flatten())


    threshold = skimage.filters.threshold_otsu(sumImage)

    imageBoundary = sumImage > threshold

    plt.plot([threshold,threshold],[0,1000])

    plt.figure()

    plt.imshow(imageBoundary,cmap=colormap)

else:
    imageBoundary = tensor[0] > -1

offset = int((convSquare-1)/2)
tensorFilt = np.zeros((len(tensor),nrows-2*offset,ncols-2*offset))
newImageBoundary = np.zeros((nrows-2*offset,ncols-2*offset))
for r in range(offset,nrows-offset):
    for c in range(offset,ncols-offset):
        newImageBoundary[r-offset:r+offset+1,c-offset:c+offset+1] = imageBoundary[r,c]
        for t in range(len(tensor)):
            tempMat = tensor[t,r-offset:r+offset+1,c-offset:c+offset+1]
            coef = imageBoundary[r-offset:r+offset+1,c-offset:c+offset+1]
            coef = coef / max([1,np.sum(coef)])
            tensorFilt[t,r-offset,c-offset] = np.sum(np.multiply(tempMat,coef))

normalizeTensor(tensorFilt)

index = 0
for t in tensorFilt:
    im = Image.fromarray(t)
    im.save(filename.replace(".txt","") + "_" + formula + "_isotopolgue_m+" + str(index) + '.tif')
    index+=1
    plt.imshow(t,cmap=colormap)
    plt.colorbar()
    plt.title("m+" + str(index))
    plt.figure()